In [3]:
import time
import random
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import IPython
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import PolynomialFeatures,LabelEncoder
import xgboost as xgb
import catboost as cbt
import lightgbm as lgb
from itertools import product
from scipy.stats import chi,entropy,rankdata
# from sklearn.model import BayesianRidge
# from sklearn.linear_model import LogisticRegression,BayesianRidge,LassoLars,ARDRegression,Ridge,SGDRegressor,ElasticNet,Lars,LassoLarsIC,HuberRegressor,RANSACRegressor,TheilSenRegressor,TweedieRegressor,
# PoissonRegressor,GammaRegressor,PassiveAggressiveRegressor
from sklearn.linear_model import LogisticRegression,BayesianRidge,Ridge,SGDRegressor,HuberRegressor
from sklearn.ensemble import RandomForestClassifier
# TweedieRegressor


def list_all_files(rootdir):
    _files = []
    #列出文件夹下所有的目录与文件
    list_file = os.listdir(rootdir)
    for i in range(0,len(list_file)):
        # 构造路径
        path = os.path.join(rootdir,list_file[i])
        # 判断路径是否是一个文件目录或者文件
        # 如果是文件目录，继续递归
        if os.path.isdir(path):
            _files.extend(list_all_files(path))
        if os.path.isfile(path) and path.split('.')[-1] == 'csv' and (path.split('.')[-2]).split('_')[-1] != 'sqrt' and (path.split('/')[-1]).split('_')[-1] != 'baseline.csv':
            _files.append(path)
    return _files

def list_all_npy(rootdir):
    _files = []
    #列出文件夹下所有的目录与文件
    list_file = os.listdir(rootdir)
    for i in range(0,len(list_file)):
        # 构造路径
        path = os.path.join(rootdir,list_file[i])
        # 判断路径是否是一个文件目录或者文件
        # 如果是文件目录，继续递归
        if os.path.isdir(path):
            _files.extend(list_all_npy(path))
        if os.path.isfile(path) and path.split('.')[-1] == 'npy':
            _files.append(path)
    return _files

oof_pre_files = list_all_files('sub_1')
oof_train_files = list_all_npy('sub_1')

oof_train = []
oof_predict = []
for oof_pre in oof_pre_files:
    oof_predict.append(pd.read_csv(oof_pre)['isDefault'].values)

for oof_t in oof_train_files:
    oof_train.append(np.load(oof_t))

y_train = pd.read_csv('../../data/train.csv')['isDefault']
test = pd.read_csv('../../data/testB.csv')



In [4]:
train_stack = np.vstack(oof_train).transpose()
test_stack = np.vstack(oof_predict).transpose()

k_fold = 10
# k_fold = 10
folds = KFold(n_splits=k_fold, shuffle=True, random_state=19960218)
oof_stack = np.zeros(train_stack.shape[0])
predictions_stack = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_stack, y_train)):
    print("fold n°{}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], y_train.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], y_train.iloc[val_idx].values

    print("-" * 10 + "BayesianRidge" + str(fold_) + "-" * 10)
    clf = BayesianRidge()


    clf.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf.predict(val_data)
    predictions_stack += clf.predict(test_stack) / k_fold



result_aucs = roc_auc_score(y_train,oof_stack)
print("BayesianRidge CV score: ",result_aucs)
##################################
submit = pd.read_csv('../../data/sample_submit.csv')
submit['id'] = test['id']
submit['isDefault'] = predictions_stack

submit.to_csv('5_18/'+str(k_fold)+'sub_1_stacking_BR_auc_{}.csv'.format(result_aucs), index = False)

submit.head(3)

# 0.7448498251261916
#  0.7448558073799789 0.7455

fold n°0
----------BayesianRidge0----------
fold n°1
----------BayesianRidge1----------
fold n°2
----------BayesianRidge2----------
fold n°3
----------BayesianRidge3----------
fold n°4
----------BayesianRidge4----------
fold n°5
----------BayesianRidge5----------
fold n°6
----------BayesianRidge6----------
fold n°7
----------BayesianRidge7----------
fold n°8
----------BayesianRidge8----------
fold n°9
----------BayesianRidge9----------
BayesianRidge CV score:  0.7463923582494334


,id,isDefault
0,1000000,0.413941
1,1000001,0.059874
2,1000002,0.013346
